# Simple linear regression model (by OLS)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std


np.random.seed(9876789)

In [ ]:
#Load data by using pandas read_csv("file_name_in_local_directory.csv")
df = pd.read_csv("dataset_gdp_inv.csv")

In [ ]:
df

In [ ]:
#Assigning data to variables X and Y by lables
Y = df['GDPgr']
X = df['Invest']

In [ ]:
X

In [ ]:
#This line is necessary to add intercept
x = sm.add_constant(X)

In [ ]:
x

In [ ]:
#To have a simple linear regression just call sm.OLS(dependent, independent)
model = sm.OLS(Y, x)
results = model.fit()
print(results.summary())

In [ ]:
#Regression significance check by comparing with critical a value of 0.05
def regression_check(regression):
    if regression.f_pvalue < 0.05:
        print('The regression model is statistically significant.')
    elif regression.f_pvalue >= 0.05:
        if regression.f_pvalue <=0.1:
            print('Regression is not statistically significant, but it has a tendency to be significant.')
        else:
            print("Regression is not statistically significant.")

In [ ]:
regression_check(results)

In [ ]:
print(str(round(results.rsquared*100,0))+'%', 'of changes in GDPgr changes is be described by changes in Invest')
print("The rest", str(round(100-results.rsquared*100,0))+'%', "of the changes are described by unknown factors")
print("Regression model for this data set is", "Y = " + str(round(results.params['const'],4))+' + '+str(round(results.params['Invest'],4))+'x')


In [ ]:
#Predictor(s) significance check by comparing with critical a value of 0.05
def coeff_check(regression):
    for j, val in enumerate(regression.pvalues):
        if val < 0.05:
             print('The coefficient', round(regression.params[j],4), 'is statistically significant.')
        elif val >= 0.05:
            if val <=0.1:
                print('The coefficient', round(regression.params[j],4), 'is not statistically significant, but it has a tendency to be significant.')
            else:
                print('The coefficient', round(regression.params[j],4), 'is not statistically significant.')


In [ ]:
coeff_check(results)

In [ ]:
#Manual prediction function
def regression_prediction(predictor, results):
    return round(results.params['const'],4) + (round(results.params['Invest'],4))*predictor
    

In [ ]:
print(regression_prediction(28,results))

In [ ]:
#Mass  prediction
dummy_data = [0, 1, 2, 3, 7, 12, 22, 23, 28, 30, 35]
for i in dummy_data:
    print(regression_prediction(i,results))

In [ ]:
#another way of predicting using in-built -- predict(data_to_predict) -- method
e = dummy_data  
e_dataframe = pd.DataFrame(e)    
pr_data = sm.add_constant(e_dataframe)
#print(pr_data)

print(results.predict(pr_data))

In [ ]:
#Graphical analysis

fig = sm.graphics.plot_ccpr(results, "Invest")
fig.tight_layout(pad=1.0)

# Higher order regression (polynomials)

In [ ]:
#Transform predictor data to a n-dim single arrays
numbersList = np.zeros(shape=(len(X),1))
for index, number in enumerate(X):
    numbersList[[index]] = number
numbersList

#Transform predictor to polynomial
from sklearn.preprocessing import PolynomialFeatures
polynomial_features= PolynomialFeatures(degree=2)
xp = polynomial_features.fit_transform(numbersList)
xp.shape

In [ ]:
numbersList

In [ ]:
xp

In [ ]:
model = sm.OLS(Y, xp).fit()
results_poly = model.predict(xp) 

results_poly.shape

In [ ]:
#Non-statsmodels visualisation

plt.scatter(numbersList,Y)
plt.plot(X,results_poly)

In [ ]:
print(model.summary())

In [ ]:
regression_check(model)
coeff_check(model)

In [ ]:
dummy_data_p = np.zeros(shape=(len(dummy_data),1))
for index, number in enumerate(dummy_data):
    dummy_data_p[[index]] = number
print(dummy_data_p)

polynomial_features = PolynomialFeatures(degree=2)
dummy_data_poly = polynomial_features.fit_transform(dummy_data_p)
dummy_data_poly

In [ ]:
print(model.predict(dummy_data_poly))
print(results.predict(pr_data))

# Multiple linear regression

In [ ]:
ds = pd.read_csv("multiple_dataset.csv")

In [ ]:
ds

In [ ]:
Y = ds['Y']
X1 = ds['Х1']
X2 = ds['Х2']
X3 = ds['Х3']



In [ ]:
#Joining all predictors as one column

joint_x = np.column_stack((X1, X2, X3, np.ones(len(X1))))

In [ ]:
joint_x

In [ ]:
res = sm.OLS(Y, joint_x).fit()
print(res.summary())

In [ ]:
regression_check(res)
coeff_check(res)

We want to create new magazine, we think that
1) is planned no. readers will be around 8500;
2) Only 20% out them will be males;
3) 30000 is median income;

In [ ]:
print(res.predict([8500, 20, 30000, 1]))

## Multicollinearity

Check by VIF - if greater than 5, then the explanatory variable is highly collinear with the other explanatory variables, and the parameter estimates will have large standard errors because of this, so the correctness of results is under a big question.

In [ ]:
#Checking for multicollinearity (const excluded)
print(res.condition_number)

from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = [variance_inflation_factor(joint_x, i) for i in range(joint_x.shape[1])]
print(vif[:-1])


## Influence plot

The plot identified the influential observation as 32 and 47. If exclude the 32nd and 47th case from the analysis, the slope coefficient changes as well as R2 will go up!


In [ ]:
fig = sm.graphics.influence_plot(res, criterion="cooks")
fig.tight_layout(pad=1.0)

In [ ]:
from statsmodels.graphics.regressionplots import plot_leverage_resid2
ax = figsize=(15, 5)
fig = plot_leverage_resid2(res, ax = ax)


In [ ]:
fig = sm.graphics.plot_partregress_grid(res)
fig.tight_layout(pad=1.0)

# Data standartization and standardized regression

In [ ]:
from scipy import stats
# standardizing dataframe
df_z = ds.select_dtypes(include=[np.number]).dropna().apply(stats.zscore)

df_z

ZY = df_z['Y']
ZX1 = df_z['Х1']
ZX2 = df_z['Х2']
ZX3 = df_z['Х3']

joint_zx = np.column_stack((ZX1, ZX2, ZX3, np.ones(len(ZX1))))

In [ ]:
joint_zx

In [ ]:
res = sm.OLS(ZY, joint_zx).fit()
print(res.summary())

In [ ]:
#Absotule significane (const excluded)
predictors = abs(res.params[:-1])

print(sorted(predictors))

#### Reduced Models

In [ ]:
red_joint_x = np.delete(joint_x, 1, 1)

In [ ]:
red_res = sm.OLS(Y, red_joint_x).fit()
print(red_res.summary())

In [ ]:
regression_check(red_res)
coeff_check(red_res)

In [ ]:
vif = [variance_inflation_factor(red_joint_x, i) for i in range(red_joint_x.shape[1])]
print(vif[:-1])

# Another analysis

In [ ]:
tax_data = pd.read_csv("simple_regr_tax.csv")

In [ ]:
tax_data

In [ ]:
tY = tax_data['Taxes']
tX = tax_data['Population']
tx = sm.add_constant(tX)

In [ ]:
tx

In [ ]:
model_tax = sm.OLS(tY, tx).fit()
print(model_tax.summary())

In [ ]:
regression_check(model_tax)
coeff_check(model_tax)

In [ ]:
model_tax.predict([1, 2300]) 